<img align="left" src = https://linea.org.br/wp-content/themes/LIneA/imagens/logo-header.jpg width=100 style="padding: 20px"> 

<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=160 style="padding: 20px">  

# Executing Rail with DP0.1 sample - FALTA ESCREVER NELE, mas a execução foi

**Contact author**: Heloisa da Silva Mengisztki ([heloisasmengisztki@gmail.com](mailto:heloisasmengisztki@gmail.com)) 

**Last verified run**: 2022-12-01 (YYYY-MM-DD) <br><br>

Here we are going to execute rail using a dp0.1 sample, the reason is that the dp0.1 sample has the redshift column so we can train and test the algorithm and see what we can obtain.

## Imports

In [ ]:
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rail
import qp
import tables_io

from rail.core.utils import RAILDIR
from rail.core.data import TableHandle, DataStore
from rail.core.stage import RailStage
from rail.core.utilStages import ColumnMapper, TableConverter

from rail.estimation.algos.bpz_lite import Inform_BPZ_lite, BPZ_lite

from rail.evaluation.evaluator import Evaluator

%matplotlib inline 
help(rail)

## General Configs

In [ ]:
pd.set_option('display.max_rows', 20)

In [ ]:
CURR_DIR = os.getcwd()
CURR_DIR, RAILDIR

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

## Reading DP0.1 csv

In [ ]:
interesting_headers = [
"coadd_objects_id",
"ra",
"dec",
"mag_g",
"magerr_g",
"mag_i",
"magerr_i",
"mag_r",
"magerr_r",
"mag_u",
"magerr_u",
"mag_y",
"magerr_y",
"mag_z",
"magerr_z",
"z_true"
]

trainFile = os.path.join(CURR_DIR, '../../../../DATA/dp0_train_random.csv')
full_data = pd.read_csv(trainFile, usecols=interesting_headers)

In [ ]:
size = len(full_data)//2

train = full_data.sample(n=size,random_state=1, ignore_index=True)
test = full_data.drop(train.index)

In [ ]:
train

In [ ]:
test

---

In [ ]:
columns_remmap = {
"coadd_objects_id": "id",
"ra": "coord_ra",
"dec": "coord_dec",
"mag_g": "mag_g_lsst",
"magerr_g": "mag_err_g_lsst",
"mag_i": "mag_i_lsst",
"magerr_i": "mag_err_i_lsst",
"mag_r": "mag_r_lsst",
"magerr_r": "mag_err_r_lsst",
"mag_u": "mag_u_lsst",
"magerr_u": "mag_err_u_lsst",
"mag_y": "mag_y_lsst",
"magerr_y": "mag_err_y_lsst",
"mag_z": "mag_z_lsst",
"magerr_z": "mag_err_z_lsst",
"z_true": "redshift"
}


col_remapper_train = ColumnMapper.make_stage(name='col_remapper_train', columns=columns_remmap)
table_conv_train = TableConverter.make_stage(name='table_conv_train', output_format='numpyDict')

results_remmaped = col_remapper_train(train)
train_data = table_conv_train(results_remmaped)

In [ ]:
tables_io.convertObj(train_data.data, tables_io.types.PD_DATAFRAME)

## Inform - Priors

In [ ]:
columns_file = os.path.join(CURR_DIR, '../configs/bpz.columns')
inform_bpz = Inform_BPZ_lite.make_stage(
    name='inform_bpzlite', 
    input="output_table_conv_train", #"inprogress_output_table_conv_train.hdf5", 
    model='trained_BPZ_output.pkl', ##não precisaria isso pro bpz
    hdf5_groupname='', 
    columns_file=columns_file
)
inform_bpz.config.to_dict()

In [ ]:
%%time
returned = inform_bpz.inform(train_data)

___

## Posterior - Estimate


In [ ]:
results_remmaped = col_remapper_train(test)
test_data = table_conv_train(results_remmaped)

In [ ]:
estimate_bpz = BPZ_lite.make_stage(
    name='estimate_bpz', 
    hdf5_groupname='', 
    columns_file=columns_file, 
    model=inform_bpz.get_handle('model'))

In [ ]:
bpz_estimated = estimate_bpz.estimate(test_data)

In [ ]:
bpz_estimated().build_tables()

results_tables = tables_io.convertObj(bpz_estimated().build_tables()['ancil'], tables_io.types.PD_DATAFRAME)
results_tables

## Evaluate

In [ ]:
test_data_orig = test_data.data

evaluator = Evaluator.make_stage(name=f'bpz_eval', truth=test_data_orig)
result_dict = evaluator.evaluate(bpz_estimated, test_data_orig)

In [ ]:
results_tables = tables_io.convertObj(result_dict.data, tables_io.types.PD_DATAFRAME)
results_tables.head()

___
## Resultado pz x spec-z

In [ ]:
zmode = bpz_estimated().ancil['zmode']

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(test_data()['redshift'],zmode,s=1,c='k',label='simple bpz mode')
plt.plot([0,3],[0,3],'r--');
plt.xlabel("true redshift")
plt.ylabel("bpz photo-z")

CONCLUSÃO

1. Cortou alguns pits. Por que?
2. Os pontos estão relativamente mais próximos da linha mas ainda tem bastante pontos espalhados. Por que?